# Find threshold for deepseg_sc

This notebook runs an algorithm that finds the best combination of threshold for each contrast, by minimizing the outpout CSA associated with each threshold.

In [1]:
import os
import numpy as np
import pandas as pd
import itertools

In [2]:
folder_csv = '/Volumes/projects/sct_deepseg_threshold/20191015'

thr = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # thresholds to test
contrasts = ['T1', 'T2', 'T2s', 'MTon', 'T1w', 'DWI']  # various contrasts

# Get the number of subjects from the T1 scan, which has all the subjects
df = pd.read_csv(os.path.join(folder_csv, 'csa-T1_0.0.csv'))
n_sub = len(df)

# Read the csv files across contrasts and thresholds
df = {}
for (i_thr, i_contrast) in [(x, y) for x in range(len(thr)) for y in range(len(contrasts))]:
    csv_file = os.path.join(folder_csv, 'csa-{}_{}.csv'.format(contrasts[i_contrast], thr[i_thr]))
    df[i_contrast, i_thr] = pd.read_csv(csv_file)

def retrieve_subject(dataframe):
    """Retrieve subject name from dataframe"""
    return os.path.split(df[0, 0]['Filename'][i_sub])[1].split('_')[0]

# Loop across subjects and populate csa array
csa = []
subjects = []
for i_sub in range(n_sub):
    populate = True
    # Retrieve subject name for the first contrast
    sub = retrieve_subject(df[0, 0])
    # initialize tmp dic to populate contrast and thr info for this subject
    csatmp = {}
    # Loop across threshold and contrast
    for (i_thr, i_contrast) in [(x, y) for x in range(len(thr)) for y in range(len(contrasts))]:
        # Retrieve line corresponding to subject
        dftmp = df[i_contrast, i_thr][df[i_contrast, i_thr]['Filename'].str.contains(sub)]
        if not len(dftmp) == 1:
            print('WARNING: Missing entry for i_sub: {}, thr: {}, contrast: {}'.format(i_sub, thr[i_thr], contrasts[i_contrast]))
            populate = False
            break
        else:
            try:
                # Retrieve the first element of the temporary dataframe (knowing there is only one element)
                # Note: we force it to float, because the presence of "None" in the df converts the whole df to string.
                csatmp[i_contrast, i_thr] = float(dftmp['MEAN(area)'].iloc[0])
            except ValueError as err:
                print('ValueError: {}: i_sub: {}, thr: {}, contrast: {}'.format(err, i_sub, thr[i_thr], contrasts[i_contrast]))
                populate = False
                break
    # If everything went well, populate csa variable
    if populate:
        print("Populate for i_sub: {}".format(i_sub))
        csa.append(csatmp)
        subjects.append(sub)

Populate for i_sub: 0
Populate for i_sub: 1
Populate for i_sub: 2
Populate for i_sub: 3
Populate for i_sub: 4
Populate for i_sub: 5
Populate for i_sub: 6
Populate for i_sub: 7
Populate for i_sub: 8
Populate for i_sub: 9
Populate for i_sub: 10
Populate for i_sub: 11
Populate for i_sub: 12
Populate for i_sub: 13
Populate for i_sub: 14
Populate for i_sub: 15
Populate for i_sub: 16
Populate for i_sub: 20
Populate for i_sub: 21
Populate for i_sub: 23
Populate for i_sub: 24
Populate for i_sub: 25
ValueError: could not convert string to float: 'None': i_sub: 30, thr: 0.0, contrast: T2s
Populate for i_sub: 31
Populate for i_sub: 32
Populate for i_sub: 33
Populate for i_sub: 34
Populate for i_sub: 35
Populate for i_sub: 36
Populate for i_sub: 37
Populate for i_sub: 38
Populate for i_sub: 39
Populate for i_sub: 40
Populate for i_sub: 41
Populate for i_sub: 42
Populate for i_sub: 43
Populate for i_sub: 44
Populate for i_sub: 45
Populate for i_sub: 46
Populate for i_sub: 47
Populate for i_sub: 48


In [3]:
# Update number of subjects based on those who were complete
n_sub = len(csa)

# Loop across subjects
thr_optimal = []
for i_sub in range(n_sub):
    csa_std = []
    i_csa_std = []
    # Loop across all possible permutations of thresholds and contrasts
    for i_thr in list(itertools.permutations(range(len(thr)), len(contrasts))):
        # Compute the STD of CSA across contrasts, for this particular combinations of thresholds
        csa_sub_tmp = [csa[i_sub][x, i_thr[x]] for x in range(len(contrasts))]
        csa_std.append(np.std(csa_sub_tmp))
        i_csa_std.append(i_thr)
#         print(csa_all)
    # Find the minimum across all CSA_STD, for this particular subject
    thr_optmin = [thr[x] for x in i_csa_std[np.argmin(csa_std)]]
    csa_std_optmin = np.std([csa[i_sub][x, thr.index(thr_optmin[x])] for x in range(len(contrasts))])
    csa_std_standard = np.std([csa[i_sub][x, 5] for x in range(len(contrasts))])
#     csa_optmin = [csa[i_sub][x, thr_optmin[x]] for x in range(len(contrasts))]
    thr_optimal.append(thr_optmin)

    thr_optimal_mean = np.mean(thr_optimal, axis=0)
    print("Subject: {} ({}/{})".format(subjects[i_sub], i_sub+1, n_sub))
    print("  Cumulative average optimal threshold: {}".format(thr_optimal_mean))
    print("  STD(CSA_opt): {} | STD(CSA_standard): {}".format(csa_std_optmin, csa_std_standard))

# Average the threshold per contrast
# print(thr_sub)


Subject: sub-balgrist04 (0/232)
  Cumulative average optimal threshold: [0.1 0.7 0.9 0.8 0.2 0. ]
  STD(CSA_opt): 3.160044015022056 | STD(CSA_standard): 4.497734765913282
Subject: sub-balgrist05 (1/232)
  Cumulative average optimal threshold: [0.05 0.75 0.9  0.75 0.15 0.1 ]
  STD(CSA_opt): 5.79545131894917 | STD(CSA_standard): 6.766908397731475
Subject: sub-amu02 (2/232)
  Cumulative average optimal threshold: [0.03333333 0.76666667 0.9        0.63333333 0.16666667 0.1       ]
  STD(CSA_opt): 6.903677101506566 | STD(CSA_standard): 8.014590280119851
Subject: sub-balgrist03 (3/232)
  Cumulative average optimal threshold: [0.1   0.675 0.9   0.525 0.15  0.075]
  STD(CSA_opt): 3.8731862796393095 | STD(CSA_standard): 4.989229909396465
Subject: sub-amu01 (4/232)
  Cumulative average optimal threshold: [0.12 0.7  0.9  0.56 0.14 0.06]
  STD(CSA_opt): 6.108981210225396 | STD(CSA_standard): 8.42235273213203
Subject: sub-balgrist06 (5/232)
  Cumulative average optimal threshold: [0.11666667 0.7166

Subject: sub-cardiff06 (40/232)
  Cumulative average optimal threshold: [0.11463415 0.7195122  0.89756098 0.61219512 0.17560976 0.03170732]
  STD(CSA_opt): 5.032083169485964 | STD(CSA_standard): 6.311089851352629
Subject: sub-cmrra01 (41/232)
  Cumulative average optimal threshold: [0.11666667 0.70952381 0.89761905 0.61666667 0.17380952 0.03095238]
  STD(CSA_opt): 6.293272038993505 | STD(CSA_standard): 7.517397651020208
Subject: sub-cmrra02 (42/232)
  Cumulative average optimal threshold: [0.11627907 0.71162791 0.89767442 0.61395349 0.1744186  0.03023256]
  STD(CSA_opt): 5.913911868695444 | STD(CSA_standard): 7.35114289496615
Subject: sub-cmrra03 (43/232)
  Cumulative average optimal threshold: [0.11818182 0.71363636 0.89772727 0.61590909 0.17272727 0.02954545]
  STD(CSA_opt): 5.349049052301563 | STD(CSA_standard): 6.702193397438261
Subject: sub-cmrra04 (44/232)
  Cumulative average optimal threshold: [0.12       0.70444444 0.89777778 0.62       0.17111111 0.02888889]
  STD(CSA_opt): 5

Subject: sub-mgh06 (79/232)
  Cumulative average optimal threshold: [0.14375 0.6975  0.89875 0.61    0.15125 0.02125]
  STD(CSA_opt): 4.906781761018582 | STD(CSA_standard): 6.488309522113995
Subject: sub-milan01 (80/232)
  Cumulative average optimal threshold: [0.14444444 0.7        0.89753086 0.61111111 0.15061728 0.02098765]
  STD(CSA_opt): 5.6995111180026115 | STD(CSA_standard): 6.795956410403539
Subject: sub-milan02 (81/232)
  Cumulative average optimal threshold: [0.14512195 0.70121951 0.89756098 0.61219512 0.15       0.02073171]
  STD(CSA_opt): 5.3002501429532725 | STD(CSA_standard): 6.985545631305844
Subject: sub-milan03 (82/232)
  Cumulative average optimal threshold: [0.14578313 0.70240964 0.89759036 0.61325301 0.14939759 0.02048193]
  STD(CSA_opt): 5.786241621593323 | STD(CSA_standard): 7.27706744910234
Subject: sub-milan04 (83/232)
  Cumulative average optimal threshold: [0.14642857 0.70357143 0.89761905 0.61428571 0.14880952 0.0202381 ]
  STD(CSA_opt): 4.30093409612739 | ST

Subject: sub-oxfordFmrib05 (118/232)
  Cumulative average optimal threshold: [0.15210084 0.7        0.89243697 0.62352941 0.1512605  0.01512605]
  STD(CSA_opt): 5.531720364967086 | STD(CSA_standard): 6.660273041295473
Subject: sub-oxfordFmrib06 (119/232)
  Cumulative average optimal threshold: [0.1525     0.7        0.8925     0.6225     0.15       0.01583333]
  STD(CSA_opt): 7.598751332783091 | STD(CSA_standard): 8.874258728613752
Subject: sub-oxfordFmrib07 (120/232)
  Cumulative average optimal threshold: [0.15206612 0.70082645 0.89256198 0.6231405  0.15041322 0.01570248]
  STD(CSA_opt): 5.716649605469524 | STD(CSA_standard): 7.551323281182355
Subject: sub-oxfordFmrib08 (121/232)
  Cumulative average optimal threshold: [0.15245902 0.69754098 0.89262295 0.62459016 0.15       0.01557377]
  STD(CSA_opt): 3.791352954076262 | STD(CSA_standard): 5.440843058947522
Subject: sub-oxfordFmrib09 (122/232)
  Cumulative average optimal threshold: [0.15284553 0.69837398 0.89268293 0.62357724 0.1495

Subject: sub-sherbrooke04 (157/232)
  Cumulative average optimal threshold: [0.16265823 0.70949367 0.88987342 0.62974684 0.1443038  0.01518987]
  STD(CSA_opt): 3.411718530741659 | STD(CSA_standard): 5.33539108472345
Subject: sub-sherbrooke05 (158/232)
  Cumulative average optimal threshold: [0.16289308 0.70943396 0.88993711 0.63081761 0.14402516 0.01509434]
  STD(CSA_opt): 4.191984379781156 | STD(CSA_standard): 5.760648787160925
Subject: sub-sherbrooke06 (159/232)
  Cumulative average optimal threshold: [0.1625   0.71     0.89     0.630625 0.144375 0.015   ]
  STD(CSA_opt): 4.22560852256205 | STD(CSA_standard): 5.945504188333777
Subject: sub-sherbrooke07 (160/232)
  Cumulative average optimal threshold: [0.16273292 0.70993789 0.89006211 0.63167702 0.14409938 0.01490683]
  STD(CSA_opt): 3.389313140524605 | STD(CSA_standard): 4.78177213678669
Subject: sub-stanford01 (161/232)
  Cumulative average optimal threshold: [0.16234568 0.71049383 0.89012346 0.63209877 0.14444444 0.01481481]
  STD

Subject: sub-tokyoSkyra05 (195/232)
  Cumulative average optimal threshold: [0.15918367 0.71938776 0.89081633 0.6127551  0.14846939 0.01377551]
  STD(CSA_opt): 3.693661313211612 | STD(CSA_standard): 4.801312930700289
Subject: sub-tokyoSkyra06 (196/232)
  Cumulative average optimal threshold: [0.15888325 0.71979695 0.89086294 0.61319797 0.14873096 0.01370558]
  STD(CSA_opt): 3.8280904128474966 | STD(CSA_standard): 4.996162513949813
Subject: sub-tokyoSkyra07 (197/232)
  Cumulative average optimal threshold: [0.15858586 0.71969697 0.89090909 0.61414141 0.1489899  0.01363636]
  STD(CSA_opt): 3.9746718940544334 | STD(CSA_standard): 5.22037012810009
Subject: sub-ubc05 (198/232)
  Cumulative average optimal threshold: [0.15879397 0.71758794 0.89095477 0.61306533 0.14874372 0.01356784]
  STD(CSA_opt): 5.945315996330829 | STD(CSA_standard): 7.040001407454954
Subject: sub-ubc06 (199/232)
  Cumulative average optimal threshold: [0.1585 0.7155 0.8905 0.612  0.149  0.0135]
  STD(CSA_opt): 3.9250043